# Thermal Image Enhancement Network (TEN) - IROS 2018
Thermal Image Enhancement using Convolution Neural Network

In [17]:
import torch
import torchvision
from torch import nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

import PIL.Image
from PIL import ImageFilter
import matplotlib
import matplotlib.pyplot as plt

import os

from tqdm import tqdm

%matplotlib inline

## Image Transforms

In [18]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
])

## Dataset

In [19]:
dataset_train_path = '/DATA1/chaitanya/ninad3/dataset/rgb91/'
assert(os.path.exists(dataset_train_path))

In [20]:
class rgb91(Dataset):
    
    def __init__(self,path,scale,transform=None):
        
        self.path = path
        self.scale = scale
        self.transform = transform
    
    def __len__(self):
        
        return len(os.listdir(self.path))
    
    def __getitem__(self,idx):
        
        imgList = os.listdir(self.path)
        imgName = imgList[idx]
        
        imgPath = os.path.join(self.path,imgName)
        
        img = PIL.Image.open(imgPath)
        img = img.filter(ImageFilter.GaussianBlur(radius=self.scale))
        img = img.resize((800,400),Image.BICUBIC)
        
        if self.transform is not None:
            img = self.transform(img)
       
        return (imgPath,img)

## Parameters

In [21]:
batch_size = 128
learning_rate = 0.001
num_epochs = 100
weight_decay = 5e-4

## Model

In [22]:
class ten(nn.Module):
    def __init__(self):
        super(ten, self).__init__()
        self.model = nn.Sequential(

            nn.Conv2d(1,64,7,stride=1,padding=3),
            nn.ReLU(True),

            nn.Conv2d(64,32,5,stride=1,padding=2),
            nn.ReLU(True),

            nn.Conv2d(32,32,3,stride=1,padding=1),
            nn.ReLU(True),

            nn.Conv2d(32,1,3,stride=1,padding=1)
        )

    def forward(self,x):
        x = self.model(x)
        return x

In [23]:
model = torch.nn.DataParallel(ten()).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, 
                             weight_decay=weight_decay)

## Run the model

In [ ]:
logs = open('./results/4e4d/try5/logs.txt',"w")
logs.write('Thermal Enhancement Network (TEN) - Results\n\n')

total_epoch_time = 0
losses = []

for epoch in tqdm(range(num_epochs)):
    
    start = time()
    
    
    

## Methods

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    learning_rate = learning_rate/10
    logs.write(f'Learning Rate set to {learning_rate}\n')
    print(f'Learning Rate set to {learning_rate}')
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate

In [ ]:
def save_checkpoint(state, output_dir, filename):
    torch.save(state, os.path.join(output_dir,filename))